<a href="https://colab.research.google.com/github/Firepower12/Abdul_miscellaneous/blob/main/Untitled6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
import numpy as np
from IPython.display import display, clear_output
from math import sin, cos, sqrt, atan2, radians, log
import statsmodels.api as sm # import statsmodels 

import warnings
warnings.filterwarnings('ignore')

from google.colab import output
output.enable_custom_widget_manager()

In [2]:
lndn = [
'EC1A',  'EC1M',  'EC1N',  'EC1P',  'EC1R',  'EC1V',  'EC1Y', 'EC2A',  'EC2M',  'EC2N',  'EC2P',  'EC2R',  'EC2V',  'EC2Y',  'EC3A',  'EC3M',  'EC3N',  'EC3P',  'EC3R',  'EC3V',
'EC4A',  'EC4M',  'EC4N',  'EC4R',  'EC4V',  'EC4Y', 'WC1A',  'WC1B',  'WC1E',  'WC1H',  'WC1N',  'WC1R',  'WC1V',  'WC1X', 'WC2A', 'WC2B',  'WC2E',  'WC2H',  'WC2N',  'WC2R']

for i in range(1,20):
    lndn.append('SW'+str(i+1))
    
for i in range(22):
    lndn.append('N'+str(i+1))
    
for i in range(18):
    lndn.append('E'+str(i+1))
    
for i in range(11):
    lndn.append('NW'+str(i+1))
    
for i in range(28):
    lndn.append('SE'+str(i+1))
    
for i in range(1,14):
    lndn.append('W'+str(i+1))


In [3]:
def getLatLong(outcode):
  url = f'https://api.postcodes.io/outcodes/{outcode}'
  response = requests.get(url)
  clear_output(wait=True) 
  display('Fetching ' + outcode + '...'),


  if response.status_code == 200:
    response_json = response.json()
    return {'outcode':outcode, 'latitude':response_json['result']['latitude'], 'longitude':response_json['result']['longitude']}
  else:
    print(response.status_code)
    return {'outcode': outcode, 'latitude': None, 'longitude': None}


locations = map(getLatLong, lndn)
df = pd.DataFrame(locations)

'Fetching W14...'

In [4]:
print(df)

    outcode   latitude  longitude
0      EC1A  51.520408  -0.103933
1      EC1M  51.521294  -0.102306
2      EC1N  51.519860  -0.108805
3      EC1P  51.524566  -0.112034
4      EC1R  51.524980  -0.108347
..      ...        ...        ...
146     W10  51.523130  -0.217157
147     W11  51.512278  -0.215848
148     W12  51.508333  -0.239846
149     W13  51.513020  -0.321223
150     W14  51.494671  -0.210219

[151 rows x 3 columns]


In [5]:
def distToCenter(lat,long):
    """
    input: a lat/long pair
    output: approx dist in km
    """
    R = 6373.0 #radius of earth 
    
    #input cords
    lat1,lon1 = radians(lat),radians(long)
    #center of london cords
    lat2,lon2 = radians(51.50918760717325),radians(-0.1275460113184446)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

df['distToCenter'] = list(map(distToCenter, df['latitude'], df['longitude']))

In [6]:
df

,outcode,latitude,longitude,distToCenter
0,EC1A,51.520408,-0.103933,2.056497
1,EC1M,51.521294,-0.102306,2.205826
2,EC1N,51.519860,-0.108805,1.758426
3,EC1P,51.524566,-0.112034,2.019548
4,EC1R,51.524980,-0.108347,2.202636
...,...,...,...,...
146,W10,51.523130,-0.217157,6.393578
147,W11,51.512278,-0.215848,6.122454
148,W12,51.508333,-0.239846,7.774968
149,W13,51.513020,-0.321223,13.414095


In [7]:

url2 = f'https://raw.githubusercontent.com/evanpiermont/evanpiermont.github.io/master/pc.json'
response1 = requests.get(url2)
data = json.loads(response1.text)
clear_output(wait=True) 
display('Fetching ' + '...'),




outcode2rightmove = {x['outcode'] : x['code'] for  x in data}

outcode2rightmove

def URLFromPostcode(pc):
  CODE = outcode2rightmove.get(pc.upper())
  if outcode2rightmove:
    return f'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=OUTCODE%5E{CODE}&sortType=6'
  else:
    return None






'Fetching ...'

In [8]:
from tqdm import tqdm

#for urls in URLFromPostcode(df['outcode'']):


def avg_price(pc):
  page = requests.get(URLFromPostcode(pc))
  soup = BeautifulSoup(page.content, 'html.parser')
  prices = soup.find_all("div", class_="propertyCard-priceValue")
  
  

  def price_to_integer(price_str):
    cleaned_price = price_str.replace("£", "").replace(",", "").strip()
    return int(cleaned_price)

  integer_prices = []

  for price in prices:
    #print(price.text)
    price_text = price.text.strip()
    try:
        integer_price = price_to_integer(price_text)
        integer_prices.append(integer_price)
    except ValueError:
        print(f"Couldn't convert '{price_text}' to an integer.")
  return np.mean(integer_prices)




avg_price("se17")


for i in tqdm(range(len(df))):
    pc = df.loc[i, 'outcode']
    print(f"Processing {pc}... ({i + 1}/{len(df)})")
    df.loc[i, 'AvgPrice'] = avg_price(pc)


  

  0%|          | 0/151 [00:00<?, ?it/s]

Processing EC1A... (1/151)


  1%|          | 1/151 [00:00<01:59,  1.26it/s]

Couldn't convert '' to an integer.
Processing EC1M... (2/151)


  1%|▏         | 2/151 [00:01<02:00,  1.24it/s]

...

 99%|█████████▊| 149/151 [02:20<00:01,  1.03it/s]

Processing W13... (150/151)


 99%|█████████▉| 150/151 [02:21<00:00,  1.03it/s]

Processing W14... (151/151)


100%|██████████| 151/151 [02:22<00:00,  1.06it/s]


In [9]:
df = df.dropna(subset = 'AvgPrice')
X = df['distToCenter']
y = df['AvgPrice']
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               AvgPrice   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     57.51
Date:                Mon, 22 May 2023   Prob (F-statistic):           4.13e-12
Time:                        14:11:26   Log-Likelihood:                -2087.8
No. Observations:                 143   AIC:                             4180.
Df Residuals:                     141   BIC:                             4186.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1.568e+06   8.95e+04     17.530   

In [10]:
df

,outcode,latitude,longitude,distToCenter,AvgPrice
0,EC1A,51.520408,-0.103933,2.056497,7.993958e+05
1,EC1M,51.521294,-0.102306,2.205826,8.368521e+05
2,EC1N,51.519860,-0.108805,1.758426,1.230600e+06
4,EC1R,51.524980,-0.108347,2.202636,9.280580e+05
5,EC1V,51.526812,-0.097840,2.840861,9.355980e+05
...,...,...,...,...,...
146,W10,51.523130,-0.217157,6.393578,1.400600e+06
147,W11,51.512278,-0.215848,6.122454,1.826000e+06
148,W12,51.508333,-0.239846,7.774968,8.329980e+05
149,W13,51.513020,-0.321223,13.414095,8.325440e+05


In [11]:
x_new = 4.5
x_new_with_constant = [[1, 4.5]]

predicted_avg_price = model.predict(x_new_with_constant)

print(f"Predicted AvgPrice for distToCenter = {x_new}: {predicted_avg_price[0]:.2f}")




Predicted AvgPrice for distToCenter = 4.5: 1204434.22


In [12]:
api_key = 'ENTER YOUR API KEY HERE'

!pip install gmaps
import gmaps
gmaps.configure(api_key) #replace with your API key: https://developers.google.com/maps/documentation/geocoding/get-api-key
figure_layout = { #make it look nice
    'width': '1000px',
    'height': '600px',
}
LonCenter = [51.50918760717325, -0.1275460113184446]
fig = gmaps.figure(center = (LonCenter), zoom_level=12)

heatmap_layer = gmaps.heatmap_layer(
    df[['latitude', 'longitude']], weights=df['AvgPrice'],
    max_intensity=df['AvgPrice'].max(), point_radius=20
)
fig.add_layer(heatmap_layer)
fig

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.8 MB/s eta 0:00:00
  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076091 sha256=ab350c3c30c5835cba50fd472e5765d5bb06f69d2f91d0f75581a1d0f52540ac
  Stored in directory: /root/.cache/pip/wheels/b3/c2/dc/48b3ef16c2184dae51a003f17eb5d065bbbf1af3437d9f14e3
Successfully built gmaps


Figure(layout=FigureLayout(height='420px'))